In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm, tqdm_notebook
import time, os
import lightfm as lfm
from lightfm import data
from lightfm import cross_validation
from lightfm import evaluation
from lightfm.evaluation import auc_score, precision_at_k

## Carga de datos

In [2]:
df_test = pd.read_csv('./data/ejemplo_de_solucion.csv')
df_train = pd.read_csv('./data/postulaciones/postulaciones_train.csv')
print(df_test.shape)
print(df_train.shape)

(312464, 2)
(6468689, 3)


In [3]:
df_postulante = pd.read_csv("./data/postulantes/postulantes_justInTrain.csv")
df_avisos = pd.read_csv("./data/avisos/avisos_filtrados.csv")
print(df_postulante.shape)
print(df_avisos.shape)

(283163, 5)
(18362, 7)


In [4]:
# Reemplazo los 6 NaN que hay en denominacion_empresa por vacio
df_avisos = df_avisos.replace(np.nan, '', regex=True)

In [5]:
df_postulante.head()

,idpostulante,fechanacimiento,sexo,nombre,estado
0,NM5M,1970-12-03,FEM,Secundario,Graduado
1,5awk,1962-12-04,FEM,Universitario,Graduado
2,ZaO5,1978-08-10,FEM,Terciario/Técnico,Graduado
3,NdJl,1969-05-09,MASC,Posgrado,En Curso
4,eo2p,1981-02-16,MASC,Secundario,Graduado


In [6]:
df_avisos.head()

,idaviso,titulo,nivel_laboral,nombre_area,denominacion_empresa,online_desde,online_hasta
0,8725750,VENDEDOR/A PROVINCIA DE SANTA FE,Senior / Semi-Senior,Comercial,VENTOR,2018-01-15,2018-02-10
1,17903700,Enfermeras,Senior / Semi-Senior,Salud,Farmacias Central Oeste,2018-03-20,2018-04-17
2,1000610287,CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA,Senior / Semi-Senior,Transporte,Wurth Argentina S.A,2018-01-15,2018-03-17
3,1001135716,Vendedor Viajante TUCUMAN/SANTIAGO DEL ESTERO,Senior / Semi-Senior,Ventas,Wurth Argentina S.A,2018-01-15,2018-04-16
4,1001326344,Vendedor Viajante RECONQUISTA/AVELLANEDA,Senior / Semi-Senior,Ventas,Wurth Argentina S.A,2018-01-15,2018-03-27


In [7]:
df_avisos.denominacion_empresa.unique()

array(['VENTOR', 'Farmacias Central Oeste', 'Wurth Argentina S.A', ...,
       'ISAC', 'País Marcela', 'GO-BETWEEN RRHH'], dtype=object)

In [8]:
df_postulante.head()

,idpostulante,fechanacimiento,sexo,nombre,estado
0,NM5M,1970-12-03,FEM,Secundario,Graduado
1,5awk,1962-12-04,FEM,Universitario,Graduado
2,ZaO5,1978-08-10,FEM,Terciario/Técnico,Graduado
3,NdJl,1969-05-09,MASC,Posgrado,En Curso
4,eo2p,1981-02-16,MASC,Secundario,Graduado


In [9]:
df_avisos.titulo.unique()
i_f=[]
for titulo in df_avisos.titulo.unique():
    i_f.append('titulo:'+titulo)
for nivel_laboral in df_avisos.nivel_laboral.unique():
    i_f.append('nivel_laboral:'+nivel_laboral)
for nombre_area in df_avisos.nombre_area.unique():
    i_f.append('nombre_area:'+nombre_area)
for de in df_avisos.denominacion_empresa.unique():
   i_f.append('denominacion_empresa:'+str(de))

#nombre area y nivel laboral

In [10]:
# feature_name,feature_value posibilities
# res = str(x)+ ":" +str(y)
u_f=[]
u_f.append('sexo:FEM')
u_f.append('sexo:MASC')
u_f.append('sexo:NO_DECLARA')
u_f.append('nombre:Secundario')
u_f.append('nombre:Universitario')
u_f.append('nombre:Terciario/Técnico')
u_f.append('nombre:Posgrado')
u_f.append('nombre:Otro')
u_f.append('nombre:Master')
u_f.append('nombre:Doctorado')
u_f.append('estado:Graduado')
u_f.append('estado:En Curso')
u_f.append('estado:Abandonado')
u_f
# Sexo y nombre

['sexo:FEM',
 'sexo:MASC',
 'sexo:NO_DECLARA',
 'nombre:Secundario',
 'nombre:Universitario',
 'nombre:Terciario/Técnico',
 'nombre:Posgrado',
 'nombre:Otro',
 'nombre:Master',
 'nombre:Doctorado']

In [11]:
# we call fit to supply userid, item id and user/item features 
ds = lfm.data.Dataset()
ds.fit(users=df_train['idpostulante'].unique(), # list of all the users
       items=df_avisos["idaviso"].unique(), #list of all the items
       user_features = u_f, #additional user features
       item_features = i_f) #additional item features
ds.interactions_shape()

(302787, 18362)

In [12]:
(interactions, weights) = ds.build_interactions(df_train[['idpostulante','idaviso']].itertuples(index=False))
interactions

<302787x18362 sparse matrix of type '<class 'numpy.int32'>'
	with 6468689 stored elements in COOrdinate format>

In [13]:
#interactions.todense()
#weights.todense()

In [14]:
# Creo user_tuple
uf_list=[]
for row in tqdm_notebook(df_postulante.itertuples()):
    uf=[]
    uf.append('sexo:'+row.sexo)
    uf.append('nombre:'+row.nombre)
    uf.append('estado:'+row.estado) # Probar no estado
    uf_list.append(uf)
#uf_list 
user_tuple = list(zip(df_postulante.idpostulante, uf_list)) 
user_tuple[:10]  

[('NM5M', ['sexo:FEM', 'nombre:Secundario']),
 ('5awk', ['sexo:FEM', 'nombre:Universitario']),
 ('ZaO5', ['sexo:FEM', 'nombre:Terciario/Técnico']),
 ('NdJl', ['sexo:MASC', 'nombre:Posgrado']),
 ('eo2p', ['sexo:MASC', 'nombre:Secundario']),
 ('Ez8J', ['sexo:MASC', 'nombre:Universitario']),
 ('aOQq', ['sexo:MASC', 'nombre:Universitario']),
 ('8BkL', ['sexo:FEM', 'nombre:Universitario']),
 ('1d2B', ['sexo:MASC', 'nombre:Universitario']),
 ('NPBx', ['sexo:MASC', 'nombre:Universitario'])]

In [15]:
# Creo item_tuple
if_list=[]
for row in tqdm_notebook(df_avisos.itertuples()):
    i_f=[]
    i_f.append('titulo:'+row.titulo)
    i_f.append('nivel_laboral:'+row.nivel_laboral) ###
    i_f.append('nombre_area:'+row.nombre_area) ###
    i_f.append('denominacion_empresa:'+row.denominacion_empresa)
    if_list.append(i_f)
#if_list 
item_tuple = list(zip(df_avisos.idaviso, if_list)) 
item_tuple[:2]  

[(8725750, ['nivel_laboral:Senior / Semi-Senior', 'nombre_area:Comercial']),
 (17903700, ['nivel_laboral:Senior / Semi-Senior', 'nombre_area:Salud'])]

In [16]:
user_features = ds.build_user_features(user_tuple, normalize= False)
item_features = ds.build_item_features(item_tuple, normalize= False)

In [17]:
user_id_map, user_feature_map, item_id_map, item_feature_map = ds.mapping()
#user_feature_map

In [18]:
#Modelo
# TODO: Probar no pasar sample_weight
t_0 = time.time()
model = lfm.LightFM(loss='warp')
#%time model.fit(interactions,user_features= user_features,sample_weight= weights,epochs=10,num_threads=8)
%time model.fit(interactions,user_features= user_features, item_features= item_features ,epochs=10,num_threads=8)
print ("tiempo: ", time.time() - t_0)

CPU times: user 6min 26s, sys: 1.82 s, total: 6min 27s
Wall time: 1min 1s
tiempo:  61.829225063323975


In [19]:
#print("Train precision: %.2f" % precision_at_k(model, df_train, k=5).mean())

In [20]:
#t_0 = time.time()
#train_auc = auc_score(model,
#                      interactions,
#                      user_features=user_features
#                     ).mean()
#print('Hybrid training set AUC: %s' % train_auc)
#print ("tiempo: ", time.time() - t_0)

In [21]:
# Prediccion para un usuario existente
user_x = user_id_map['1d2B']
n_users, n_items = interactions.shape # number of users * number of items
#predict_scores = model.predict(user_x, np.arange(n_items))
predict_scores = model.predict(user_x, np.arange(n_items), user_features= user_features, item_features= item_features) # means predict for all 
predict_scores

array([-715.09667969, -718.32531738, -717.6439209 , ..., -717.10015869,
       -719.5581665 , -717.55541992])

In [22]:
np.sort(predict_scores)[::-1][:10]

array([-712.70452881, -713.45288086, -713.79901123, -713.95861816,
       -713.98022461, -713.99053955, -714.0065918 , -714.04956055,
       -714.08502197, -714.10406494])

In [23]:
# Me da las posiciones donde estan los avisos de mayor score   
posiciones_scores = np.argsort(predict_scores)[::-1][:10]
posiciones_scores

array([10693,  3915,  2148,  6358,  8806,   123,  1048,  8559,  4406,
        4637])

In [24]:
items=df_avisos["idaviso"].unique()

In [25]:
items[posiciones_scores]

array([1112206678, 1112296169, 1112348918, 1112308219, 1112315589,
       1112016972, 1112278394, 1112260111, 1112356835, 1112360030])

In [26]:
df_train.loc[df_train['idpostulante']=='1d2B']

,idaviso,idpostulante,fechapostulacion
91,1112020573,1d2B,2018-02-19 07:01:34
92,1112325331,1d2B,2018-02-19 07:04:50
93,1112343340,1d2B,2018-02-26 21:54:31


In [27]:
idpost_train = df_train.idpostulante
idpost_test = df_test.idpostulante
idpost_train_set = set(idpost_train.unique().tolist())
idpost_test_set = set(idpost_test.unique().tolist())

test_unique = (idpost_test_set - idpost_train_set)
test_intersect = (idpost_test_set - test_unique)
print("unicos de test: ",len(test_unique ))
print("compartidos con train: ", len(test_intersect))

unicos de test:  41204
compartidos con train:  115028


In [28]:
users = test_intersect
users_unknown = test_unique

In [38]:
avisos_abril = set(df_avisos.loc[df_avisos['online_hasta']>='2018-04-01'].idaviso)
avisos_abril_list = df_avisos.loc[df_avisos['online_hasta']>='2018-04-01'].idaviso.tolist()
print(len(avisos_abril))
print(len(df_avisos))

6722
18362


In [30]:
user_x = user_id_map['akO24jJ']
predict_scores = model.predict(user_x, np.arange(n_items), user_features= user_features, item_features= item_features)
avisos = items[np.argsort(predict_scores)[::-1][:100]

In [31]:
1112393398 in avisos_abril

True

In [32]:
1112403164 in avisos_abril

False

In [33]:
# black list de avisos (avisos que estan en Train y no quiero que aparezcan en las predicciones)
#avisos_train_bl = df_train.drop_duplicates(subset='idaviso')
#avisos_train_bl = set(avisos_train_bl.idaviso)

In [33]:
def enAbril(ranking_list,avisos_abril, avisos):
    for i in avisos:
        if (i in avisos_abril) :
            ranking_list.append(i)
        if (len(ranking_list)==10):
            break
    return ranking_list

In [34]:
avisos_dummy=[]
enAbril(avisos_dummy,avisos_abril, avisos)

[1111644641,
 1112320618,
 1112033906,
 1112334788,
 1112205158,
 1112208702,
 1112339238,
 1112334791,
 1112208550,
 1112208757]

In [35]:
# TODO: Si el PRONOSTICADO ya esta en TRAIN, hay que SACARLO!!!
scored_list=[]
for user in tqdm_notebook(users):
    top_avisos=[]
    user_x = user_id_map[user]
    predict_scores = model.predict(user_x, np.arange(n_items), user_features= user_features, item_features= item_features)    
    avisos_full = items[np.argsort(predict_scores)[::-1]]
    #Validos solo los que estan en ABRIL.
    enAbril(top_avisos, avisos_abril, avisos_full)
    for aviso in top_avisos:
        user_aviso=dict()
        user_aviso["idaviso"] = aviso
        user_aviso["idpostulante"] = user
        scored_list.append(user_aviso)

#scored_list

In [36]:
# Creo dataframe con la lista scoreada de los postulantes que estan en train y test
df_modelo = pd.DataFrame(scored_list)
df_modelo.head()

,idaviso,idpostulante
0,1112147486,5QMBew
1,1112248723,5QMBew
2,1112406827,5QMBew
3,1112413366,5QMBew
4,1112306543,5QMBew


In [39]:
# Random con AVISOS DE ABRIL 2018 y no en Train
scored_list_unknow = []
for user_unknown in tqdm_notebook(users_unknown):
    for index in range(10):
        user_aviso=dict()
        user_aviso["idaviso"] = random.choices(avisos_abril_list)[0]
        user_aviso["idpostulante"] = user_unknown
        scored_list_unknow.append(user_aviso)
#scored_list_unknow

In [40]:
# Creo dataframe con la lista random de los postulantes que estan solo en test
df_modelo_unknow = pd.DataFrame(scored_list_unknow)
df_modelo_unknow.head()

,idaviso,idpostulante
0,1112376988,X9oXkmO
1,1112016155,X9oXkmO
2,1112447747,X9oXkmO
3,1112408354,X9oXkmO
4,1112444435,X9oXkmO


In [41]:
# Appendeo al modelo creado con lightFM para los postulantes de ambos ds los postulantes del ds Testing unicamente..
df_modeloligthFM = df_modelo.append(df_modelo_unknow)
df_modeloligthFM.head()

,idaviso,idpostulante
0,1112147486,5QMBew
1,1112248723,5QMBew
2,1112406827,5QMBew
3,1112413366,5QMBew
4,1112306543,5QMBew


In [42]:
# Imprimo longitud del dataset final.
print(len(df_modeloligthFM.idpostulante))
print(len(df_modeloligthFM.idpostulante.unique()))
print(len(df_modeloligthFM.idaviso.unique()))

1562320
156232
6722


In [43]:
#modeloligthFM_1                     ---> Resultado Kaggle: 0.00028
#modeloligthFM_2   UF                ---> Resultado Kaggle: 0.00077   (2500 idaviso.unique)
#modeloligthFM_3   UF e IF           ---> Resultado Kaggle: 0.00135   (5449 idaviso.unique)
#modeloligthFM_4   UF+IF+Abril       ---> Resultado Kaggle: 0.00216   (6722 idaviso.unique)
#modeloligthFM_5   UF'+IF'+Abril-Train ---> Resultado Kaggle: 0.00177 (6722 idaviso.unique)
#modeloligthFM_5   UF+IF+Abril-Train ---> Resultado Kaggle:  ????         (  ???  idaviso.unique)

df_modeloligthFM.to_csv("./submissions/modeloligthFM_5.csv", index=False)